# Interactive exploration of networks

This notebook demonstrates the interactive exploration capabilities of PyPSA networks.

In [ ]:
import pypsa

In [ ]:
n = pypsa.Network("../networks/scigrid-de/scigrid-de.nc")

In [ ]:
import numpy as np
import pandas as pd

rng = np.random.default_rng()
bus_sizes = pd.Series(rng.integers(200, 30000, size=len(n.buses)), index=n.buses.index)
dict_bus_sizes = dict(zip(bus_sizes.index, bus_sizes))

In [ ]:
# n.explore(bus_sizes=bus_sizes)
n.explore(bus_sizes=dict_bus_sizes)

In [ ]:
# Alternatively
from pypsa.plot.maps.interactive import PydeckPlotter

plotter = PydeckPlotter(n)
print(plotter)
print(plotter._layers)
print("Adding bus layer with sizes...")
plotter.add_bus_layer(bus_sizes=bus_sizes)
print(plotter._layers.keys())
print("Showing the plot...")
plotter.show()

In [ ]:
n.optimi